### Bronze layer

In [1]:
import duckdb
import os
import glob

# --- 1. CONFIGURATION & CONSTANTS ---
RAW_DATA_PATH = '../data/raw'
LAKEHOUSE_PATH = '../data/lakehouse'
METADATA_PATH = os.path.join(LAKEHOUSE_PATH, 'metadata.duckdb')

# Official Data Source URLs
URL_MITMA = "https://www.transportes.gob.es/ministerio/proyectos-singulares/estudios-de-movilidad-con-big-data/opendata-movilidad"
URL_INE = "https://www.ine.es/"
URL_CNIG = "https://centrodedescargas.cnig.es/CentroDescargas/index.jsp"
URL_MTDFP = "https://datos.gob.es/es/catalogo/l01280796-calendario-laboral"

# --- 2. INFRASTRUCTURE FUNCTION ---

def init_lakehouse():
    """
    Initializes the DuckDB connection, loads the DuckLake extension, 
    attaches the persistent catalog, and ensures logical schemas exist.
    """
    print("--- 🔌 Initializing Lakehouse Infrastructure ---")
    
    # Ensure physical directory exists
    os.makedirs(LAKEHOUSE_PATH, exist_ok=True)
    
    # Create an in-memory compute session
    con = duckdb.connect(database=':memory:')
    
    # 1. Install & Load DuckLake Extension
    try:
        con.execute("INSTALL ducklake;")
        con.execute("LOAD ducklake;")
    except Exception as e:
        print(f"❌ Error loading 'ducklake': {e}")
        raise e

    # 2. Attach the Catalog (Metadata Layer)
    # This separates compute (RAM) from storage (Disk)
    con.execute(f"ATTACH 'ducklake:{METADATA_PATH}' AS lakehouse")
    
    # 3. Ensure Logical Schemas Exist
    for schema in ['bronze', 'silver', 'gold']:
        con.execute(f"CREATE SCHEMA IF NOT EXISTS lakehouse.{schema}")
        
    print(f"✅ Lakehouse catalog attached at: {METADATA_PATH}")
    return con

# --- 3. INGESTION FUNCTIONS ---

def ingest_mobility_bronze(con):
    """
    Ingests the massive Mobility Fact Table using a Partitioning Strategy (by Date).
    Pattern: Create Structure -> Configure Partitioning -> Insert Data.
    """
    print("\n--- 🚚 Ingesting Mobility Data (Fact Table) ---")
    
    # Locate all compressed CSV files for municipalities
    mitma_files = glob.glob(os.path.join(RAW_DATA_PATH, 'mitma', '*_Viajes_municipios.csv.gz'))
    
    if not mitma_files:
        print("❌ No mobility files found!")
        return

    # Step 1: Create empty table structure (LIMIT 0)
    # This defines the schema without processing all data yet.
    print("-> Creating table structure...")
    con.execute(f"""
        CREATE OR REPLACE TABLE lakehouse.bronze.mobility_sample_week AS
        SELECT 
            *,
            CURRENT_TIMESTAMP AS ingestion_timestamp,
            '{URL_MITMA}' AS source_url
        FROM read_csv_auto({mitma_files}, filename=true, all_varchar=true)
        LIMIT 0;
    """)
    
    # Step 2: Configure Partitioning in DuckLake
    # This ensures data is physically organized by folders (e.g., /fecha=20230101/)
    print("-> Configuring partition strategy (by date)...")
    con.execute("""
        ALTER TABLE lakehouse.bronze.mobility_sample_week 
        SET PARTITIONED BY (fecha);
    """)
    
    # Step 3: Insert Massive Data
    print("-> Inserting data (this may take a while)...")
    con.execute(f"""
        INSERT INTO lakehouse.bronze.mobility_sample_week 
        SELECT 
            *,
            CURRENT_TIMESTAMP AS ingestion_timestamp,
            '{URL_MITMA}' AS source_url
        FROM read_csv_auto({mitma_files}, filename=true, all_varchar=true);
    """)
    
    print("✅ Successfully ingested: lakehouse.bronze.mobility_sample_week")

def ingest_dimensions_bronze(con):
    """
    Ingests all auxiliary dictionary tables using a helper function 
    to handle different CSV formats (separators, encoding).
    """
    print("\n--- 📚 Ingesting Dimension Tables ---")

    # Inner helper function to avoid code repetition
    def ingest_one(table, filename, url, folder='mitma', sep=';', encoding='utf-8', **kwargs):
        path = os.path.join(RAW_DATA_PATH, folder, filename)
        if not os.path.exists(path):
            print(f"⚠️ Missing file: {filename}")
            return

        # Construct dynamic options for read_csv_auto
        options = [f"all_varchar=true", f"sep='{sep}'", f"encoding='{encoding}'"]
        for k, v in kwargs.items():
            # Handle boolean SQL syntax conversion (True -> 'true')
            val = str(v).lower() if isinstance(v, bool) else f"'{v}'"
            options.append(f"{k}={val}")
        
        options_str = ", ".join(options)
        
        # Escape single quotes in URL for SQL safety
        safe_url = url.replace("'", "''")

        # Execute CTAS (Create Table As Select)
        con.execute(f"""
            CREATE OR REPLACE TABLE lakehouse.bronze.{table} AS
            SELECT 
                *,
                CURRENT_TIMESTAMP AS ingestion_timestamp,
                '{safe_url}' AS source_url
            FROM read_csv_auto('{path}', {options_str});
        """)
        print(f"✅ Ingested: lakehouse.bronze.{table}")

    # --- INGESTION LIST ---
    
    # 1. MITMA Files (Using Pipe '|' separator)
    ingest_one('zoning_municipalities', 'nombres_municipios.csv', URL_MITMA, sep='|', header=True)
    ingest_one('population_municipalities', 'poblacion_municipios.csv', URL_MITMA, sep='|')
    ingest_one('mapping_ine_mitma', 'relacion_ine_zonificacionMitma.csv', URL_MITMA, sep='|')
    
    # 2. INE / CNIG Files (Using Semicolon ';' separator)
    ingest_one('ine_rent_municipalities', 'ine_renta.csv', URL_INE, folder='ine', sep=';')
    ingest_one('municipal_coordinates', 'municipios_coordenadas.csv', URL_CNIG, folder='ine', sep=';')
    ingest_one('work_calendars', 'calendario.csv', URL_MTDFP, folder='ine', sep=';')

# --- 4. VALIDATION FUNCTION ---

def validate_bronze(con):
    """
    Performs a quick check to verify that tables exist in the Bronze schema.
    """
    print("\n--- 🔍 Checking Bronze Layer Status ---")
    # We query the internal system metadata to see what tables are registered
    res = con.execute("""
        SELECT table_name 
        FROM duckdb_tables() 
        WHERE schema_name = 'bronze'
    """).df()
    print(res)

In [ ]:
# --- 5. MAIN EXECUTION PIPELINE ---

def run_bronze_pipeline():
    """
    Orchestrates the entire Bronze Layer process.
    """
    con = init_lakehouse()          # 1. Initialize Infrastructure
    ingest_mobility_bronze(con)     # 2. Load Mobility Data (Partitioned)
    ingest_dimensions_bronze(con)   # 3. Load Dimension Tables
    validate_bronze(con)            # 4. Final Validation
    return con

# Execute only if running as a script/notebook
if __name__ == "__main__":
    con = run_bronze_pipeline()

### Silver layer

In [2]:
# --- 1. SILVER LAYER FUNCTIONS ---

def build_silver_dim_zones(con):
    """
    Construye la dimensión de zonas (dim_zones) limpiando y deduplicando.
    """
    print("\n--- Building Silver Table: dim_zones ---")
    
    query = """
    CREATE OR REPLACE TABLE lakehouse.silver.dim_zones AS
    WITH unique_mapping AS (
        SELECT DISTINCT 
            CAST(municipio_mitma AS VARCHAR) as mitma_ref,
            MIN(CAST(municipio_ine AS VARCHAR)) as ine_ref
        FROM lakehouse.bronze.mapping_ine_mitma
        WHERE municipio_mitma IS NOT NULL
        GROUP BY municipio_mitma
    ),
    raw_zones AS (
        SELECT 
            TRIM(z.ID) AS mitma_code,
            CASE 
                WHEN TRIM(m.ine_ref) = 'NA' THEN NULL 
                ELSE TRIM(m.ine_ref) 
            END AS ine_code,
            TRIM(z.name) AS zone_name
        FROM lakehouse.bronze.zoning_municipalities z
        LEFT JOIN unique_mapping m 
            ON TRIM(z.ID) = TRIM(m.mitma_ref)
        WHERE z.ID IS NOT NULL AND z.ID != 'ID'
        GROUP BY z.ID, z.name, m.ine_ref
    )
    SELECT 
        ROW_NUMBER() OVER (ORDER BY mitma_code) AS zone_id,
        mitma_code,
        ine_code,
        zone_name,
        'municipal' AS zone_level,
        CURRENT_TIMESTAMP AS processed_at
    FROM raw_zones
    ORDER BY zone_id;
    """
    con.execute(query)
    print("✅ Created: lakehouse.silver.dim_zones")

def build_silver_metric_population(con):
    """
    Limpia y estructura los datos de población.
    """
    print("\n--- Building Silver Table: metric_population ---")
    
    query = """
    CREATE OR REPLACE TABLE lakehouse.silver.metric_population AS
    SELECT 
        z.zone_id,
        CAST(TRY_CAST(column1 AS DOUBLE) AS BIGINT) AS population,
        '2023' AS year,
        CURRENT_TIMESTAMP AS processed_at
    FROM lakehouse.bronze.population_municipalities p
    JOIN lakehouse.silver.dim_zones z ON TRIM(p.column0) = z.mitma_code
    WHERE column0 IS NOT NULL 
      AND NOT regexp_matches(column1, '[a-zA-Z]')
    """
    con.execute(query)
    print("✅ Created: lakehouse.silver.metric_population")

def build_silver_dim_coordinates(con):
    """
    Limpia las coordenadas geográficas.
    """
    print("\n--- Building Silver Table: dim_coordinates ---")
    
    query = """
    CREATE OR REPLACE TABLE lakehouse.silver.dim_coordinates AS
    SELECT 
        z.zone_id,
        TRY_CAST(REPLACE(c.LATITUD_ETRS89, ',', '.') AS DOUBLE) AS latitude,
        TRY_CAST(REPLACE(c.LONGITUD_ETRS89, ',', '.') AS DOUBLE) AS longitude,
        CURRENT_TIMESTAMP AS processed_at
    FROM lakehouse.bronze.municipal_coordinates c
    JOIN lakehouse.silver.dim_zones z 
        ON LEFT(c.COD_INE, 5) = z.ine_code 
    WHERE z.zone_id IS NOT NULL;
    """
    con.execute(query)
    print("✅ Created: lakehouse.silver.dim_coordinates")

def build_silver_metric_ine_rent(con):
    """
    Limpia los datos de renta media del INE.
    """
    print("\n--- Building Silver Table: metric_ine_rent ---")
    
    query = """
    CREATE OR REPLACE TABLE lakehouse.silver.metric_ine_rent AS
    SELECT 
        z.zone_id,
        CAST(TRY_CAST(REPLACE(r.Total, '.', '') AS DOUBLE) AS BIGINT) AS income_per_capita,
        CAST(r.Periodo AS INTEGER) AS year,
        CURRENT_TIMESTAMP AS processed_at
    FROM lakehouse.bronze.ine_rent_municipalities r
    JOIN lakehouse.silver.dim_zones z 
        ON split_part(r.Municipios, ' ', 1) = z.ine_code
    WHERE r."Indicadores de renta media" = 'Renta neta media por persona'
      AND (r.Distritos IS NULL OR r.Distritos = '')
      AND (r.Secciones IS NULL OR r.Secciones = '')
      AND r.Total IS NOT NULL
      AND z.zone_id IS NOT NULL;
    """
    con.execute(query)
    print("✅ Created: lakehouse.silver.metric_ine_rent")

def build_silver_fact_mobility(con):
    """
    Creating the mobility fact table (Fact Table).
    """
    print("\n--- Building Silver Table: fact_mobility ---")
    
    # 1. Create Schema
    schema_query = """
    CREATE OR REPLACE TABLE lakehouse.silver.fact_mobility (
        date DATE,
        hour INTEGER,
        origin_zone_id BIGINT,
        destination_zone_id BIGINT,
        trips DOUBLE,
        trips_km DOUBLE,
        processed_at TIMESTAMP
    );
    """
    con.execute(schema_query)
    
    # 2. Insert Data (Sorted for performance instead of PARTITIONED BY)
    print("-> Inserting data...")
    insert_query = """
    INSERT INTO lakehouse.silver.fact_mobility
    SELECT 
        strptime(fecha, '%Y%m%d') AS date,
        CAST(periodo AS INTEGER) AS hour,
        zo.zone_id AS origin_zone_id,
        zd.zone_id AS destination_zone_id,
        TRY_CAST(REPLACE(REPLACE(viajes, '.', ''), ',', '.') AS DOUBLE) AS trips,
        TRY_CAST(REPLACE(REPLACE(viajes_km, '.', ''), ',', '.') AS DOUBLE) AS trips_km,
        CURRENT_TIMESTAMP AS processed_at
    FROM lakehouse.bronze.mobility_sample_week m
    LEFT JOIN lakehouse.silver.dim_zones zo ON TRIM(m.origen) = zo.mitma_code
    LEFT JOIN lakehouse.silver.dim_zones zd ON TRIM(m.destino) = zd.mitma_code
    WHERE viajes IS NOT NULL
    ORDER BY date;
    """
    con.execute(insert_query)
    print("✅ Created: lakehouse.silver.fact_mobility")

def build_silver_dim_festive_types(con):
    """
    Normaliza los tipos de festivos.
    """
    print("\n--- Building Silver Table: dim_festive_types ---")
    
    query = """
    CREATE OR REPLACE TABLE lakehouse.silver.dim_festive_types AS
    SELECT DISTINCT
        CASE 
            WHEN "Tipo de Festivo" ILIKE '%festivo nacional%' OR "Tipo de Festivo" ILIKE '%fiesta nacional%'
            THEN 'NationalFestive'
            ELSE "Tipo de Festivo"
        END AS festive_type,
        CURRENT_TIMESTAMP AS processed_at
    FROM lakehouse.bronze.work_calendars
    WHERE "Tipo de Festivo" IS NOT NULL
      AND ("Tipo de Festivo" ILIKE '%festivo nacional%' OR "Tipo de Festivo" ILIKE '%fiesta nacional%');
    """
    con.execute(query)
    print("✅ Created: lakehouse.silver.dim_festive_types")

def build_silver_bridge_zones_festives(con):
    """
    Crea el puente entre zonas y días festivos.
    """
    print("\n--- Building Silver Table: bridge_zones_festives ---")
    
    query = """
    CREATE OR REPLACE TABLE lakehouse.silver.bridge_zones_festives AS
    SELECT z.zone_id,
        CAST(strptime(c."Dia", '%d/%m/%Y') AS DATE) AS festive_date,
        ft.festive_type,
        CURRENT_TIMESTAMP AS processed_at
    FROM lakehouse.silver.dim_zones AS z
    CROSS JOIN lakehouse.bronze.work_calendars AS c
    JOIN lakehouse.silver.dim_festive_types AS ft
    ON ft.festive_type =
        CASE WHEN c."Tipo de Festivo" ILIKE '%festivo nacional%' OR c."Tipo de Festivo" ILIKE '%fiesta nacional%'
            THEN 'NationalFestive'
            ELSE c."Tipo de Festivo"
        END;
    """
    con.execute(query)
    print("✅ Created: lakehouse.silver.bridge_zones_festives")

def validate_silver_layer(con):
    """
    Verifica que las tablas se han creado.
    """
    print("\n--- 🔍 Checking Silver Layer Status ---")
    res = con.execute("""
        SELECT table_name 
        FROM duckdb_tables() 
        WHERE database_name = 'lakehouse' AND schema_name = 'silver';
    """).df()
    print(res)

#### Orchestrate Bronze Layer and Silver Layer

In [3]:
import time

def run_full_data_pipeline():
    """
    MASTER PIPELINE: Orchestrates the full End-to-End ETL process (Bronze -> Silver).
    This function simulates a DAG run (e.g., Airflow) by executing tasks in dependency order.
    """
    start_time = time.time()
    print("🚀 STARTING DUCKLAKE PIPELINE 🚀")
    print("="*60)

    # --- STEP 0: INFRASTRUCTURE INITIALIZATION ---
    # We initialize the DuckDB session, load extensions, and attach the Catalog.
    con = init_lakehouse()

    # --- STEP 1: BRONZE LAYER (Raw Ingestion) ---
    print("\n" + "="*20 + " STAGE 1: BRONZE LAYER " + "="*20)
    
    # 1.1 Ingest Massive Data (Fact Table)
    # This handles the partitioning and bulk load of mobility data.
    ingest_mobility_bronze(con)
    
    # 1.2 Ingest Reference Data (Dimensions)
    # Loads dictionaries (INE, MITMA, Calendars, Coordinates).
    ingest_dimensions_bronze(con)
    
    # 1.3 Validation
    validate_bronze(con)

    # --- STEP 2: SILVER LAYER (Cleaning & Star Schema) ---
    print("\n" + "="*20 + " STAGE 2: SILVER LAYER " + "="*20)
    
    # 2.1 Core Dimension (CRITICAL DEPENDENCY)
    # We MUST build 'dim_zones' first because all other tables JOIN with it.
    build_silver_dim_zones(con)
    
    # 2.2 Enriched Dimensions & Metrics
    # These tables depend on 'dim_zones' but can run in parallel with each other.
    build_silver_metric_population(con)
    build_silver_metric_ine_rent(con)
    build_silver_dim_coordinates(con)
    build_silver_dim_festive_types(con)
    
    # 2.3 Bridge Tables & Fact Tables
    # These run last because they rely on the dimensions created above.
    build_silver_bridge_zones_festives(con)
    build_silver_fact_mobility(con)
    
    # 2.4 Validation
    validate_silver_layer(con)

    # --- FINAL SUMMARY ---
    elapsed = time.time() - start_time
    print("\n" + "="*60)
    print(f"✅ PIPELINE FINISHED SUCCESSFULLY in {elapsed:.2f} seconds.")
    print("="*60)
    
    # We return the active connection so you can query the results interactively
    return con

# --- EXECUTION ---
if __name__ == "__main__":
    # Run the pipeline and store the connection in 'con'
    con = run_full_data_pipeline()

🚀 STARTING DUCKLAKE PIPELINE 🚀
--- 🔌 Initializing Lakehouse Infrastructure ---
✅ Lakehouse catalog attached at: ../data/lakehouse\metadata.duckdb

==================== STAGE 1: BRONZE LAYER ====================

--- 🚚 Ingesting Mobility Data (Fact Table) ---
-> Creating table structure...
-> Configuring partition strategy (by date)...
-> Inserting data (this may take a while)...
✅ Successfully ingested: lakehouse.bronze.mobility_sample_week

--- 📚 Ingesting Dimension Tables ---
✅ Ingested: lakehouse.bronze.zoning_municipalities
✅ Ingested: lakehouse.bronze.population_municipalities
✅ Ingested: lakehouse.bronze.mapping_ine_mitma
✅ Ingested: lakehouse.bronze.ine_rent_municipalities
✅ Ingested: lakehouse.bronze.municipal_coordinates
✅ Ingested: lakehouse.bronze.work_calendars

--- 🔍 Checking Bronze Layer Status ---
                  table_name
0    ine_rent_municipalities
1      zoning_municipalities
2       mobility_sample_week
3  population_municipalities
4      municipal_coordinates
5 